# Coinbase API Overview
---

### Imports

In [50]:
import datetime
import logging
import random
import sys
import time

from elasticsearch import Elasticsearch

# A hacky way of (temporarily) adding the root folder to PATH
if "../" not in sys.path:
    sys.path.insert(0, "../")

from marketdata import MarketData
from api.cbexchange import Coinbase
from api.logs.setuplogger import logger

# Allow Jupyter to output variables on separate lines
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Logging Setup
This can be accomplished from any module in the root folder. Once I figure out how to do absolute imports properly it should be doable from anywhere in the project. 

In [8]:
from api.logs.setuplogger import logger
LOG = logging.getLogger('root.docs.coinbase.api.overview')
LOG.debug('started...')

`DEBUG 2019-07-14 13:12:18,060 ~ root.docs.coinbase.api.overview.<module>(): started...`


## Coinbase - Base Module
---

In [58]:
from api.cbexchange import Coinbase
cb = Coinbase()
cb.server_time()

{'epoch': 1563093609.913, 'iso': '2019-07-14T08:40:09.913Z'}

### Get an up-to date list of available trade pairs 

In [55]:
available_products = cb.valid_product_ids()
available_products[:7]
len(available_products)

('LINK-ETH', 'ZEC-BTC', 'BAT-ETH', 'ETH-DAI', 'ETH-EUR', 'ETH-USD', 'LTC-EUR')

48

### Check the latest trade (ticker) for ETH-USD

In [53]:
product_id = available_products[5]
cb.ticker(product_id)

{'ask': '266.69',
 'bid': '266.63',
 'price': '266.63000000',
 'size': '0.90195599',
 'time': '2019-07-14T08:37:38.350Z',
 'trade_id': 50009878,
 'volume': '72024.28963447'}

### Get the latest trades for ETH-USD

In [81]:
trades = cb.trades(product_id)
len(trades)
trades[:2]

100

[{'price': '266.42000000',
  'side': 'buy',
  'size': '0.90865349',
  'time': '2019-07-14T08:55:00.668Z',
  'trade_id': 50009967},
 {'price': '266.42000000',
  'side': 'buy',
  'size': '2.00000000',
  'time': '2019-07-14T08:55:00.668Z',
  'trade_id': 50009966}]

### Get candles for a certain time period

In [75]:
granularities = cb.available_granularity()
granularities.values

[60, 300, 900, 3600, 21600, 86400]

In [79]:
start = datetime.datetime(year=2019, month=7, day=1)
end = datetime.datetime(year=2019, month=7, day=5)

In [76]:
candles = cb.candles(product_id=product_id, start=start, end=end, granularity=granularities.max())
candles

[[1562284800, 280, 294.9, 282.81, 288, 106439.56862999],
 [1562198400, 280.1, 304, 301.77, 282.8, 100227.78726344],
 [1562112000, 289.61, 303.83, 291.25, 301.79, 105705.70508639],
 [1562025600, 270.42, 296.24, 293.72, 291.25, 221624.50052172],
 [1561939200, 278.65, 301.32, 290.08, 293.7, 184839.74588246]]

### Get the best bid and ask

In [82]:
cb.order_book(product_id)

{'asks': [['265.64', '3.14854682', 1]],
 'bids': [['265.63', '28.05851611', 5]],
 'sequence': 6995150409}

### Get top 50 asks and bids (aggregated)

In [89]:
ob = cb.order_book(product_id, level=2)
len(ob['asks'])
len(ob['bids'])
ob

50

50

{'asks': [['265.37', '102.155', 5],
  ['265.4', '10', 1],
  ['265.58', '0.75', 1],
  ['265.63', '13.27791753', 1],
  ['265.68', '5.72949016', 1],
  ['265.71', '22.06', 1],
  ['265.74', '0.03628013', 1],
  ['265.77', '18.38669267', 1],
  ['265.78', '99.83150064', 2],
  ['265.82', '29.29413063', 2],
  ['265.95', '100', 1],
  ['265.96', '20.56', 1],
  ['265.99', '13.60539236', 1],
  ['266.04', '26.5276845', 2],
  ['266.09', '22.55', 1],
  ['266.11', '16.82717406', 1],
  ['266.14', '7.51484', 1],
  ['266.24', '18.1', 1],
  ['266.27', '0.12', 1],
  ['266.28', '16.6310225', 1],
  ['266.35', '3.76236878', 1],
  ['266.36', '24.56', 1],
  ['266.48', '62.77579931', 1],
  ['266.49', '189.58958038', 1],
  ['266.69', '1', 1],
  ['266.75', '45.06', 1],
  ['266.76', '67.41853151', 1],
  ['266.85', '7.51484', 1],
  ['266.86', '3.75065636', 1],
  ['266.97', '3.75840944', 1],
  ['267.21', '25.70537403', 1],
  ['267.28', '0.0269887', 1],
  ['267.29', '0.29930039', 1],
  ['267.41', '1.001', 1],
  ['267.63

### Get the _entire_ orderbook

In [87]:
ob = cb.order_book(product_id, level=3)
len(ob['asks'])
len(ob['bids'])
ob

21841

12833

{'asks': [['264.98', '0.09', '911db63d-8e19-4a14-be0e-a9243d4758d3'],
  ['264.99', '1.90993042', '8789f2a0-7e4e-4219-afe7-b0b8248f3a6a'],
  ['264.99', '0.09006958', '5a12302e-3743-41c8-8f16-77c5aa493188'],
  ['265', '4.99923469', '5df7873d-ef31-48f9-a611-8dbfeb02d511'],
  ['265.08', '10', 'd9403d3e-87e8-4b10-88a7-686e42b0e5a6'],
  ['265.12', '30.061', '9558d99a-ce07-4cff-963f-73223f2d6cc8'],
  ['265.16', '0.75', 'e2e37116-55dc-425b-9b3c-6d84c9ce4af0'],
  ['265.18', '16.28728094', '5e4df0c3-765f-437a-8d3a-3ad701595062'],
  ['265.34', '14.31223243', '8222a6ce-ce9e-4eb5-8c05-668f9179d89d'],
  ['265.37', '0.03628013', '9652c3ac-aa1e-49fd-90fc-531ebbe0060e'],
  ['265.38', '13.5', '99062b8e-ed41-48ac-a6be-ef5fa425e4d4'],
  ['265.49', '4.018', 'd99fd6fc-7fd6-4322-830d-4a75893a5ecc'],
  ['265.52', '17.3', 'a7797d09-be8f-4d22-a333-81cdf48069e9'],
  ['265.53', '25', '604357f2-2a1c-46b7-bab7-87058a190ff0'],
  ['265.63', '26.33', '33c91022-d527-406e-951a-4b888fd1a3f3'],
  ['265.63', '5.69413063', 